# Notebook for the training of the Teresa Robot
## Importing all necessary tools

In [1]:
from src.gym_envs.RobotEnv import RobotEnv # Training environment
import numpy as np
import tensorflow.compat.v1 as tf
import roslibpy # API of ROS
from src.robots.Teresa import Teresa # This is the representation of Teresa Robot
from src.utils.training_tools import NB_STATES

tf.disable_v2_behavior()

:0: UserWarning: You do not have a working installation of the service_identity module: 'No module named 'service_identity''.  Please install it from <https://pypi.python.org/pypi/service_identity> and make sure all of its dependencies are satisfied.  Without the service_identity module, Twisted can perform only rudimentary TLS client hostname verification.  Many valid certificate/hostname mappings may be rejected.


Instructions for updating:
non-resource variables are not supported in the long term


2021-07-26 16:13:28,896  WARNING: From /home/saadubuntu/Documents/robot_controller/venv/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


## Creating the connection with ROS

In [2]:
HOST = 'localhost'
PORT = 9090

client = roslibpy.Ros(host=HOST, port=PORT)


## Creating representation of the robot and introducing it in the Gym Environment

## Testing the environment

In [3]:
# client.connect()
client.run() # This run the main loop of ROS
teresa_controller = Teresa(client) # Robot API
env = RobotEnv(teresa_controller, client) # Training Environment

env.reset() # Restarting the environment to the initial state
import time
for i in range(20):
    state, reward, done, _ = env.step(np.random.randint(4))
    print(state)
    env.render()
    if done and reward:
        print("Body detected and centered")
        #env.reset()
    elif done:
        print("Face not detected. End of the episode")
        #env.reset()


2021-07-26 16:13:29,038     INFO: Connection to ROS MASTER ready.


2382
Body detected and centered
0
2383
Body detected and centered
0
0
0
2499
Body detected and centered
2376
Body detected and centered
0
2372
Body detected and centered
2264
Body detected and centered
2380
Body detected and centered
3198
0
2387
Body detected and centered
2626
Body detected and centered
2388
Body detected and centered
2274
Body detected and centered
0
0


## Controlling the Robot Manually

In [5]:
client.run()
teresa_controller = Teresa(client)
env = RobotEnv(teresa_controller, client)

env.reset()
finish = False

while not finish:
    movement = input('Enter a movement (0 Right, 1 Left, 2 Backward, 3 Forward, exit): ')
    if movement == 'exit':
        finish = True
        continue
    movement = int(movement)
    state, reward, done, _ = env.step(movement)
    if done and reward:
        print("Centered")
        #env.reset()
    env.render()

Enter a movement (0 Right, 1 Left, 2 Backward, 3 Forward, exit): 2
Enter a movement (0 Right, 1 Left, 2 Backward, 3 Forward, exit): 2
Enter a movement (0 Right, 1 Left, 2 Backward, 3 Forward, exit): 1
Enter a movement (0 Right, 1 Left, 2 Backward, 3 Forward, exit): 0
Enter a movement (0 Right, 1 Left, 2 Backward, 3 Forward, exit): 0
Enter a movement (0 Right, 1 Left, 2 Backward, 3 Forward, exit): 0
Enter a movement (0 Right, 1 Left, 2 Backward, 3 Forward, exit): 0
Enter a movement (0 Right, 1 Left, 2 Backward, 3 Forward, exit): 0
Centered
Enter a movement (0 Right, 1 Left, 2 Backward, 3 Forward, exit): 0
Centered
Enter a movement (0 Right, 1 Left, 2 Backward, 3 Forward, exit): 0
Enter a movement (0 Right, 1 Left, 2 Backward, 3 Forward, exit): 0
Enter a movement (0 Right, 1 Left, 2 Backward, 3 Forward, exit): 0
Centered
Enter a movement (0 Right, 1 Left, 2 Backward, 3 Forward, exit): 1
Enter a movement (0 Right, 1 Left, 2 Backward, 3 Forward, exit): 2
Enter a movement (0 Right, 1 Left, 

## Testing the filter haarcascade

In [5]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

face_cascade = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('Haarcascades/haarcascade_eye.xml')
body_cascade = cv2.CascadeClassifier('Haarcascades/haarcascade_fullbody.xml')

img = cv2.imread('env_observation')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(gray, 1.2, 3)
body = body_cascade.detectMultiScale(gray,1.2,3)

if len(body)>0:
    print("body detected")
    x, y, w, h = body[0]
    cv2.rectangle(img, (x, y), (x+w, y+h), (25, 125, 225), 5)

print("ok")
for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

print('ok')

cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

ok
ok


In [6]:
faces

()

## Necessary functions for the training (Neural Network Set up)

In [7]:
learning_rate = 0.01

state_size =  NB_STATES # 800 is the image size this maybe variable
action_size = 4
new_graph = tf.Graph()
## TRAINING Hyperparameters

initializer=tf.initializers.glorot_uniform()

def discount_correct_rewards(r, gamma=0.99):
  """ take 1D float array of rewards and compute discounted reward """
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    #if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add

  discounted_r -= discounted_r.mean()
  discounted_r /- discounted_r.std()
  return discounted_r

def discount_and_normalize_rewards(episode_rewards):
    discounted_episode_rewards = np.zeros_like(episode_rewards)
    cumulative = 0.0
    #print("len episode rewards",episode_rewards)
    for i in reversed(range(len(episode_rewards))):
        cumulative = cumulative * gamma + episode_rewards[i]
        #print("dans boucle",episode_rewards[i],"cyl",cumulative)
        discounted_episode_rewards[i] = cumulative
    
    mean = np.mean(discounted_episode_rewards)
    std = np.std(discounted_episode_rewards)
    if std :
        discounted_episode_rewards = (discounted_episode_rewards - mean) / (std)
    else:
        discounted_episode=[]
        discounted_episode_rewards[0] = np.array(mean)
        print("ATTTTTTTTTTTTTTTTTT")
    #print("dis",discounted_episode_rewards,"std",std)
    
    return discounted_episode_rewards

with tf.name_scope("inputs"):
    input_ = tf.placeholder(tf.float32, [None, state_size], name="input_")
    actions = tf.placeholder(tf.int32, [None, action_size], name="actions")
    discounted_episode_rewards_ = tf.placeholder(tf.float32, [None,], name="discounted_episode_rewards")
    
    # Add this placeholder for having this variable in tensorboard
    mean_reward_ = tf.placeholder(tf.float32 , name="mean_reward")

    with tf.name_scope("fc1"):
        fc1 = tf.layers.dense(input_ , 20, activation=tf.nn.relu,kernel_initializer=initializer)

    with tf.name_scope("fc2"):
        fc2 = tf.layers.dense(fc1, action_size,activation= tf.nn.relu, kernel_initializer=initializer)
    
    with tf.name_scope("fc3"):
        fc3 = tf.layers.dense(fc2, action_size, activation= None,kernel_initializer=initializer)

    with tf.name_scope("softmax"):
        action_distribution = tf.nn.softmax(fc3)

    with tf.name_scope("loss"):
        # tf.nn.softmax_cross_entropy_with_logits computes the cross entropy of the result after applying the softmax function
        # If you have single-class labels, where an object can only belong to one class, you might now consider using 
        # tf.nn.sparse_softmax_cross_entropy_with_logits so that you don't have to convert your labels to a dense one-hot array. 
        neg_log_prob = tf.nn.softmax_cross_entropy_with_logits_v2(logits = fc3, labels = actions)
        #loss = tf.nn.sparse_softmax_cross_entropy_with_logits (neg_log_prob * discounted_episode_rewards_)
        loss = tf.reduce_mean(neg_log_prob * discounted_episode_rewards_) 
        
    
    with tf.name_scope("train"):
        train_opt = tf.train.AdamOptimizer(learning_rate).minimize(loss)
# Setup TensorBoard Writer

## Losses
tf.summary.scalar("Loss", loss)

## Reward mean
tf.summary.scalar("Reward_mean", mean_reward_)

merged_summary_op = tf.summary.merge_all() #procedure d'affichage groupée dans tensorboard

Instructions for updating:
Use keras.layers.Dense instead.


2021-07-26 16:14:34,171  WARNING: From <ipython-input-7-ed69ceeb7041>:53: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.Dense instead.


Instructions for updating:
Please use `layer.__call__` method instead.


2021-07-26 16:14:34,206  WARNING: From /home/saadubuntu/Documents/robot_controller/venv/lib/python3.6/site-packages/tensorflow/python/layers/core.py:187: Layer.apply (from tensorflow.python.keras.engine.base_layer_v1) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.


## Training

In [10]:
# Parameters for the training
max_episodes = 1000
gamma = 0.95 # Discount rate
#max_batch = NbStat*5
max_batch = 20
episodes_succeded = 0

client.run() # This run the main loop of ROS
teresa_controller = Teresa(client) # Robot API
env = RobotEnv(teresa_controller, client) # Training Environment

env.reset() # Restarting the environment to the initial state
write_op = tf.summary.merge_all()
allRewards = []
total_rewards = 0
maximumRewardRecorded = 0
episode = 0
episode_states, episode_actions, episode_rewards = [],[],[]

saver = tf.train.Saver()
NbStat = state_size

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    #writer = tf.summary.FileWriter("./tensorboard/pg/1",sess.graph)
     
    for episode in range(max_episodes):
        
        episode_rewards_sum = 0

        # Launch the game
        state = env.reset()
        print(NbStat)
        ne_state=np.identity(NbStat)[state:state+1]
        episode_length=0
        while True:
            episode_length+=1
            if episode_length > max_batch:
                print ("tooooooooo long")
                break      
            # Choose action a, remember WE'RE NOT IN A DETERMINISTIC ENVIRONMENT, WE'RE OUTPUT PROBABILITIES.
            #state=int(state)
            #print("state",state,NbStat)
            ne_state=np.identity(NbStat)[state:state+1]
            action_probability_distribution = sess.run(action_distribution, feed_dict={input_: ne_state.reshape([1,NbStat])})
            action = np.random.choice(range(action_probability_distribution.shape[1]), p=action_probability_distribution.ravel())  # select action w.r.t the actions prob

            # Perform a
            ext=False
            #print("in actor mstep",state,"real ibn self")
            new_state, reward, done, info = env.step(action)
            env.render()
            #print("after action state",new_state,NbStat)
            #print(" drz actor mstep",state,"real ibn self",new_state,"rew",reward,"done",done)
            # Store s, a, r
            episode_states.append(ne_state)
                        
            # For actions because we output only one (the index) we need 2 (1 is for the action taken)
            # We need [0., 1.] (if we take right) not just the index
            action_ = np.zeros((action_size), dtype=int)
            action_[action] = 1
            
            #print("action proba",action_probability_distribution,"st",state,"new",new_state)
            state = new_state
            episode_actions.append(action_)
            
            episode_rewards.append(reward)
            if done:
                # Calculate sum reward
                if reward == 1:
                    episodes_succeded += 1
                
                episode_rewards_sum = np.sum(episode_rewards)/episode_length  # HA addded the sum 
                print(action_probability_distribution,"action proba",episode_rewards_sum,"length",episode_length,"rew",reward)
                allRewards.append(episode_rewards_sum)
                
                total_rewards = np.sum(allRewards)
                
                # Mean reward
                mean_reward = np.divide(total_rewards, episode+1)
                
                
                maximumRewardRecorded = np.amax(allRewards)
                
                print("==========================================")
                print("Episode: ", episode,"length",episode_length)
                print("Reward: ", episode_rewards_sum)
                print("Mean Reward","val",mean_reward)
                print("Max reward so far: ", maximumRewardRecorded)
                
                # Calculate discounted reward
                discounted_episode_rewards = discount_and_normalize_rewards(episode_rewards)
                
                #print("disco",discounted_episode_rewards)               
                # Feedforward, gradient and backpropagation
                #loss_, _ = sess.run([loss, train_opt], feed_dict={input_: np.vstack(np.array(episode_states)),
                #                                                 actions: np.vstack(np.array(episode_actions)),
                #                                                 discounted_episode_rewards_: discounted_episode_rewards 
                #                                                })
                loss_, _ = sess.run([loss, train_opt], feed_dict={input_: np.vstack(np.array(episode_states)),
                                                                 actions: np.vstack(np.array(episode_actions)),
                                                                 discounted_episode_rewards_: discounted_episode_rewards
                                                                })
                
 
                                                                 
                # Write TF Summaries
                summary = sess.run(write_op, feed_dict={input_: np.vstack(np.array(episode_states)),actions: np.vstack(np.array(episode_actions)),discounted_episode_rewards_: discounted_episode_rewards,
                                                                    mean_reward_: mean_reward  })
                
               
#                 writer.add_summary(summary, episode)
#                 writer.flush()
                
            
                
                # Reset the transition stores
                episode_states, episode_actions, episode_rewards = [],[],[]
                
                break
            
            
            
        # Save Model
    saver.save(sess, "pgpendul.ckpt")
    print("Model saved")
    print(episodes_succeded / max_episodes)

3596
[[0.25069818 0.24877483 0.2518319  0.24869514]] action proba 1.0 length 1 rew 1
Episode:  0 length 1
Reward:  1.0
Mean Reward val 1.0
Max reward so far:  1.0
ATTTTTTTTTTTTTTTTTT
3596
tooooooooo long
3596
[[0.25091654 0.24450013 0.26015297 0.24443042]] action proba 0.5 length 2 rew 1
Episode:  2 length 2
Reward:  0.5
Mean Reward val 0.5
Max reward so far:  1.0
3596
tooooooooo long
3596
[[0.2500287  0.247163   0.25679266 0.24601565]] action proba 0.5 length 2 rew 1
Episode:  4 length 2
Reward:  0.5
Mean Reward val 0.4
Max reward so far:  1.0
3596
tooooooooo long
3596
[[0.253955   0.238918   0.26895535 0.23817164]] action proba 0.3333333333333333 length 3 rew 1
Episode:  6 length 3
Reward:  0.3333333333333333
Mean Reward val 0.33333333333333337
Max reward so far:  1.0
3596
[[0.25545165 0.23627448 0.273248   0.23502587]] action proba 0.5 length 2 rew 1
Episode:  7 length 2
Reward:  0.5
Mean Reward val 0.3541666666666667
Max reward so far:  1.0
3596
[[0.2559951  0.2333299  0.27615508 0

[[0.249998   0.21952622 0.27892077 0.25155506]] action proba 0.0 length 8 rew 0
Episode:  48 length 8
Reward:  0.0
Mean Reward val 0.3230295781316189
Max reward so far:  1.0
ATTTTTTTTTTTTTTTTTT
3596
[[0.24905664 0.22023782 0.27776673 0.25293878]] action proba 0.25 length 4 rew 1
Episode:  49 length 4
Reward:  0.25
Mean Reward val 0.3215689865689865
Max reward so far:  1.0
3596
[[0.24754581 0.2216472  0.27576366 0.25504324]] action proba 0.0625 length 16 rew 1
Episode:  50 length 16
Reward:  0.0625
Mean Reward val 0.3164892025186143
Max reward so far:  1.0
3596
tooooooooo long
3596
[[0.24704558 0.22077695 0.2774444  0.25473312]] action proba 0.5 length 2 rew 1
Episode:  52 length 2
Reward:  0.5
Mean Reward val 0.3139801760084779
Max reward so far:  1.0
3596
[[0.24606873 0.2214078  0.27698404 0.25553942]] action proba 0.5 length 2 rew 1
Episode:  53 length 2
Reward:  0.5
Mean Reward val 0.31742498756387644
Max reward so far:  1.0
3596
tooooooooo long
3596
tooooooooo long
3596
[[0.2453899

[[0.22109756 0.23577453 0.2740438  0.26908404]] action proba 0.0 length 6 rew 0
Episode:  99 length 6
Reward:  0.0
Mean Reward val 0.33282615995115994
Max reward so far:  1.0
ATTTTTTTTTTTTTTTTTT
3596
[[0.22052553 0.23603131 0.27394214 0.26950106]] action proba 0.5 length 2 rew 1
Episode:  100 length 2
Reward:  0.5
Mean Reward val 0.334481346486297
Max reward so far:  1.0
3596
tooooooooo long
3596
[[0.22001082 0.23575231 0.27435398 0.2698829 ]] action proba 0.5 length 2 rew 1
Episode:  102 length 2
Reward:  0.5
Mean Reward val 0.3328409319914174
Max reward so far:  1.0
3596
[[0.219467   0.23547922 0.2748664  0.2701873 ]] action proba 0.3333333333333333 length 3 rew 1
Episode:  103 length 3
Reward:  0.3333333333333333
Mean Reward val 0.33284566661970505
Max reward so far:  1.0
3596
[[0.2189732  0.23522979 0.27508894 0.270708  ]] action proba 0.5 length 2 rew 1
Episode:  104 length 2
Reward:  0.5
Mean Reward val 0.3344376126518983
Max reward so far:  1.0
3596
[[0.21853459 0.2350125  0.275

tooooooooo long
3596
[[0.21792524 0.23043719 0.2877439  0.2638936 ]] action proba 0.3333333333333333 length 3 rew 1
Episode:  146 length 3
Reward:  0.3333333333333333
Mean Reward val 0.3354043632104856
Max reward so far:  1.0
3596
tooooooooo long
3596
[[0.21732248 0.23057988 0.2869869  0.26511073]] action proba 0.1 length 10 rew 1
Episode:  148 length 10
Reward:  0.1
Mean Reward val 0.33157343216068047
Max reward so far:  1.0
3596
tooooooooo long
3596
[[0.22101718 0.22958846 0.2886424  0.2607519 ]] action proba 0.07142857142857142 length 14 rew 1
Episode:  150 length 14
Reward:  0.07142857142857142
Mean Reward val 0.32765476796933746
Max reward so far:  1.0
3596
[[0.22237816 0.22912598 0.28910106 0.2593948 ]] action proba 0.5 length 2 rew 1
Episode:  151 length 2
Reward:  0.5
Mean Reward val 0.3287886181800655
Max reward so far:  1.0
3596
[[0.22330269 0.22873938 0.29017282 0.2577852 ]] action proba 0.3333333333333333 length 3 rew 1
Episode:  152 length 3
Reward:  0.3333333333333333
Mea

[[0.2292791  0.20461613 0.2890843  0.27702048]] action proba 0.3333333333333333 length 3 rew 1
Episode:  186 length 3
Reward:  0.3333333333333333
Mean Reward val 0.355046535460974
Max reward so far:  1.0
3596
[[0.23479764 0.20029294 0.29134044 0.27356902]] action proba 1.0 length 1 rew 1
Episode:  187 length 1
Reward:  1.0
Mean Reward val 0.358477138995756
Max reward so far:  1.0
ATTTTTTTTTTTTTTTTTT
3596
[[0.23592138 0.1985411  0.29097068 0.27456686]] action proba 0.5 length 2 rew 1
Episode:  188 length 2
Reward:  0.5
Mean Reward val 0.3592259372021277
Max reward so far:  1.0
3596
[[0.2357642  0.1973295  0.29037756 0.2765287 ]] action proba 1.0 length 1 rew 1
Episode:  189 length 1
Reward:  1.0
Mean Reward val 0.3625984322694849
Max reward so far:  1.0
ATTTTTTTTTTTTTTTTTT
3596
[[0.23494178 0.19631676 0.2892659  0.27947554]] action proba 0.3333333333333333 length 3 rew 1
Episode:  190 length 3
Reward:  0.3333333333333333
Mean Reward val 0.3624452118562067
Max reward so far:  1.0
3596
[[

...

[[0.09032926 0.13154985 0.35365254 0.42446843]] action proba 0.5 length 2 rew 1
Episode:  795 length 2
Reward:  0.5
Mean Reward val 0.40201488422907045
Max reward so far:  1.0
3596
[[0.08995128 0.13119039 0.35368267 0.42517573]] action proba 1.0 length 1 rew 1
Episode:  796 length 1
Reward:  1.0
Mean Reward val 0.40276517923003774
Max reward so far:  1.0
ATTTTTTTTTTTTTTTTTT
3596
[[0.0895646  0.13078186 0.3544262  0.42522725]] action proba 1.0 length 1 rew 1
Episode:  797 length 1
Reward:  1.0
Mean Reward val 0.4035135937924061
Max reward so far:  1.0
ATTTTTTTTTTTTTTTTTT
3596
tooooooooo long
3596
tooooooooo long
3596
[[0.08915509 0.13030735 0.3546083  0.4259293 ]] action proba 0.5 length 2 rew 1
Episode:  800 length 2
Reward:  0.5
Mean Reward val 0.40262652664961307
Max reward so far:  1.0
3596
[[0.08878158 0.1298689  0.35487947 0.42647004]] action proba 0.25 length 4 rew 1
Episode:  801 length 4
Reward:  0.25
Mean Reward val 0.4024362192597757
Max reward so far:  1.0
3596
[[0.08844831 

[[0.08549648 0.12603164 0.38412425 0.4043476 ]] action proba 0.5 length 2 rew 1
Episode:  841 length 2
Reward:  0.5
Mean Reward val 0.40338351971961506
Max reward so far:  1.0
3596
tooooooooo long
3596
[[0.08535531 0.12595211 0.3853802  0.40331236]] action proba 1.0 length 1 rew 1
Episode:  843 length 1
Reward:  1.0
Mean Reward val 0.4036124687250188
Max reward so far:  1.0
3596
tooooooooo long
3596
[[0.08522778 0.12600367 0.38641137 0.4023572 ]] action proba 0.0625 length 16 rew 1
Episode:  845 length 16
Reward:  0.0625
Mean Reward val 0.4027321792008462
Max reward so far:  1.0
3596
[[0.08510746 0.12604263 0.38746393 0.40138596]] action proba 1.0 length 1 rew 1
Episode:  846 length 1
Reward:  1.0
Mean Reward val 0.40343733601406834
Max reward so far:  1.0
ATTTTTTTTTTTTTTTTTT
3596
tooooooooo long
3596
[[0.0849489  0.12598668 0.38911045 0.39995396]] action proba 1.0 length 1 rew 1
Episode:  848 length 1
Reward:  1.0
Mean Reward val 0.40366480989860526
Max reward so far:  1.0
3596
[[0.08

tooooooooo long
3596
tooooooooo long
3596
[[0.07929492 0.11648433 0.41563863 0.38858214]] action proba 1.0 length 1 rew 1
Episode:  887 length 1
Reward:  1.0
Mean Reward val 0.4095330989390587
Max reward so far:  1.0
3596
tooooooooo long
3596
tooooooooo long
3596
[[0.07918455 0.11628962 0.41651985 0.38800594]] action proba 0.5 length 2 rew 1
Episode:  890 length 2
Reward:  0.5
Mean Reward val 0.40871536684386545
Max reward so far:  1.0
3596
[[0.07908102 0.11610286 0.4172174  0.3875987 ]] action proba 1.0 length 1 rew 1
Episode:  891 length 1
Reward:  1.0
Mean Reward val 0.40937824199314365
Max reward so far:  1.0
ATTTTTTTTTTTTTTTTTT
3596
[[0.07894675 0.11586138 0.41726077 0.3879311 ]] action proba 0.14285714285714285 length 7 rew 1
Episode:  892 length 7
Reward:  0.14285714285714285
Mean Reward val 0.40907978611505186
Max reward so far:  1.0
3596
[[0.07882735 0.11564629 0.41707847 0.38844794]] action proba 1.0 length 1 rew 1
Episode:  893 length 1
Reward:  1.0
Mean Reward val 0.4097407

tooooooooo long
3596
[[0.07594159 0.11314151 0.42065677 0.39026016]] action proba 0.5 length 2 rew 1
Episode:  937 length 2
Reward:  0.5
Mean Reward val 0.4058135675326424
Max reward so far:  1.0
3596
[[0.07592013 0.11323111 0.42064017 0.39020854]] action proba 1.0 length 1 rew 1
Episode:  938 length 1
Reward:  1.0
Mean Reward val 0.40644635393569606
Max reward so far:  1.0
ATTTTTTTTTTTTTTTTTT
3596
[[0.07586192 0.11324309 0.4200278  0.3908672 ]] action proba 1.0 length 1 rew 1
Episode:  939 length 1
Reward:  1.0
Mean Reward val 0.4070777939847006
Max reward so far:  1.0
ATTTTTTTTTTTTTTTTTT
3596
[[0.07576233 0.11317264 0.42015997 0.39090508]] action proba 0.5 length 2 rew 1
Episode:  940 length 2
Reward:  0.5
Mean Reward val 0.40717654234390926
Max reward so far:  1.0
3596
[[0.075642   0.11280826 0.42076913 0.39078057]] action proba 0.0 length 19 rew 0
Episode:  941 length 19
Reward:  0.0
Mean Reward val 0.4067442954836715
Max reward so far:  1.0
ATTTTTTTTTTTTTTTTTT
3596
[[0.07553364 0.

[[0.07103029 0.10221981 0.41452992 0.41221994]] action proba 1.0 length 1 rew 1
Episode:  981 length 1
Reward:  1.0
Mean Reward val 0.40911844890204585
Max reward so far:  1.0
ATTTTTTTTTTTTTTTTTT
3596
[[0.07123502 0.10192756 0.41551062 0.41132668]] action proba 1.0 length 1 rew 1
Episode:  982 length 1
Reward:  1.0
Mean Reward val 0.4097195491574863
Max reward so far:  1.0
ATTTTTTTTTTTTTTTTTT
3596
[[0.07138005 0.1016006  0.41708568 0.40993375]] action proba 1.0 length 1 rew 1
Episode:  983 length 1
Reward:  1.0
Mean Reward val 0.4103194276644401
Max reward so far:  1.0
ATTTTTTTTTTTTTTTTTT
3596
[[0.07147267 0.10124541 0.4179046  0.40937737]] action proba 1.0 length 1 rew 1
Episode:  984 length 1
Reward:  1.0
Mean Reward val 0.4109180881439686
Max reward so far:  1.0
ATTTTTTTTTTTTTTTTTT
3596
[[0.07151584 0.10086247 0.41933042 0.40829125]] action proba 1.0 length 1 rew 1
Episode:  985 length 1
Reward:  1.0
Mean Reward val 0.41151553430203763
Max reward so far:  1.0
ATTTTTTTTTTTTTTTTTT
359

In [11]:
print(episodes_succeded)

708


## Testing the Neural Network

### Structure of the NN

In [9]:
 
NbStat = NB_STATES
state_size = NbStat
action_size = 4
# new_graph = tf.Graph()
initializer=tf.initializers.glorot_uniform()
learning_rate = 0.01

def discount_correct_rewards(r, gamma=0.99):
  """ take 1D float array of rewards and compute discounted reward """
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    #if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add

  discounted_r -= discounted_r.mean()
  discounted_r /- discounted_r.std()
  return discounted_r

# Setup TensorBoard Writer
def discount_and_normalize_rewards(episode_rewards):
    discounted_episode_rewards = np.zeros_like(episode_rewards)
    cumulative = 0.0
    #print("len episode rewards",episode_rewards)
    for i in reversed(range(len(episode_rewards))):
        cumulative = cumulative * gamma + episode_rewards[i]
        #print("dans boucle",episode_rewards[i],"cyl",cumulative)
        discounted_episode_rewards[i] = cumulative
    
    mean = np.mean(discounted_episode_rewards)
    std = np.std(discounted_episode_rewards)
    if std :
        discounted_episode_rewards = (discounted_episode_rewards - mean) / (std)
    else:
        discounted_episode=[]
        discounted_episode_rewards[0] = np.array(mean)
        print("ATTTTTTTTTTTTTTTTTT")
    #print("dis",discounted_episode_rewards,"std",std)
    
    return discounted_episode_rewards
def discount_and_normalize_rewards(episode_rewards):
    discounted_episode_rewards = np.zeros_like(episode_rewards)
    cumulative = 0.0
    #print("len episode rewards",episode_rewards)
    for i in reversed(range(len(episode_rewards))):
        cumulative = cumulative * gamma + episode_rewards[i]
        #print("dans boucle",episode_rewards[i],"cyl",cumulative)
        discounted_episode_rewards[i] = cumulative
    
    mean = np.mean(discounted_episode_rewards)
    std = np.std(discounted_episode_rewards)
    if std :
        discounted_episode_rewards = (discounted_episode_rewards - mean) / (std)
    else:
        discounted_episode=[]
        discounted_episode_rewards[0] = np.array(mean)
        print("ATTTTTTTTTTTTTTTTTT")
    #print("dis",discounted_episode_rewards,"std",std)
    
    return discounted_episode_rewards

with tf.name_scope("inputs"):
    input_ = tf.placeholder(tf.float32, [None, state_size], name="input_")
    actions = tf.placeholder(tf.int32, [None, action_size], name="actions")
    discounted_episode_rewards_ = tf.placeholder(tf.float32, [None,], name="discounted_episode_rewards")
    
    # Add this placeholder for having this variable in tensorboard
    mean_reward_ = tf.placeholder(tf.float32 , name="mean_reward")

    with tf.name_scope("fc1"):
        fc1 = tf.layers.dense(input_ , 20, activation=tf.nn.relu,kernel_initializer=initializer)

    with tf.name_scope("fc2"):
        fc2 = tf.layers.dense(fc1, action_size,activation= tf.nn.relu, kernel_initializer=initializer)
    
    with tf.name_scope("fc3"):
        fc3 = tf.layers.dense(fc2, action_size, activation= None,kernel_initializer=initializer)

    with tf.name_scope("softmax"):
        action_distribution = tf.nn.softmax(fc3)

    with tf.name_scope("loss"):
        # tf.nn.softmax_cross_entropy_with_logits computes the cross entropy of the result after applying the softmax function
        # If you have single-class labels, where an object can only belong to one class, you might now consider using 
        # tf.nn.sparse_softmax_cross_entropy_with_logits so that you don't have to convert your labels to a dense one-hot array. 
        neg_log_prob = tf.nn.softmax_cross_entropy_with_logits_v2(logits = fc3, labels = actions)
        #loss = tf.nn.sparse_softmax_cross_entropy_with_logits (neg_log_prob * discounted_episode_rewards_)
        loss = tf.reduce_mean(neg_log_prob * discounted_episode_rewards_) 
        
    
    with tf.name_scope("test"):
        train_opt = tf.train.AdamOptimizer(learning_rate).minimize(loss)


## Losses
## TRAINING Hyperparameters

# tf.summary.scalar("Loss", loss)

# ## Reward mean
# tf.summary.scalar("Reward_mean", mean_reward_)

max_episodes = 500

gamma = 0.95 # Discount rate
max_batch = NbStat*5
    
episode_rewards_sum = 0

        # Launch the game
    #state = env.reset()
    #ne_state=np.identity(NbStat)[state:state+1]
    #env.render()
episode_length=0

### Running the NN

In [10]:

saver = tf.train.Saver()

client.run() # This run the main loop of ROS
teresa_controller = Teresa(client) # Robot API
env = RobotEnv(teresa_controller, client) # Training Environment

with tf.Session() as sess:
    # sess.run(tf.global_variables_initializer())
   
        # Load the model
    print("yo")
    print(saver.restore(sess, "pgpendul.ckpt"))
    if not saver.restore(sess, "pgpendul.ckpt"):
        print('yo')
    total_rewards = 0
    for episode in range(50):
        state = env.reset()
        #ne_state=np.identity(NbStat)[state:state+1]
        step = 0
        done = False
        
        print("****************************************************")
        print("EPISODE ", episode)

       
        #while True:
        j = 0
        #The Q-Network
        while j < 500:
            j+=1
            state=int(state)
            ne_state=np.identity(NbStat)[state:state+1]
            # Choose action a, remember WE'RE NOT IN A DETERMINISTIC ENVIRONMENT, WE'RE OUTPUT PROBABILITIES.
            action_probability_distribution = sess.run(action_distribution, feed_dict={input_: ne_state.reshape([1,NbStat])})
            print(action_probability_distribution)
            action = np.random.choice(range(action_probability_distribution.shape[1]), p=action_probability_distribution.ravel())  # select action w.r.t the actions prob
            #action = np.argmax(action_probability_distribution)
            

            # new_state, reward, done, info = env.step(int(action),True)
            new_state, reward, done, info = env.step(int(action))

            print("state",state,"ne_state",new_state,"action",action) 
            total_rewards += reward
            env.render()
            if done:    
                #rewards.append(total_rewards)
                print ("Score", total_rewards)
                break
            state = new_state
    env.close()
print ("Score over time: " ,  total_rewards)

yo
INFO:tensorflow:Restoring parameters from pgpendul.ckpt


2021-07-15 13:56:53,182     INFO: Restoring parameters from pgpendul.ckpt


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key dense_3/bias not found in checkpoint
	 [[node save_1/RestoreV2 (defined at /home/saadubuntu/Documents/robot_controller/venv/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'save_1/RestoreV2':
  File "/home/saadubuntu/Documents/robot_controller/venv/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/saadubuntu/Documents/robot_controller/venv/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/saadubuntu/Documents/robot_controller/venv/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/saadubuntu/Documents/robot_controller/venv/lib/python